### Bayesian Time Series Forecasting with Uber’s Orbit Package

> An easy-to-follow guide of benchmarking Bayesian models to forecast univariate time series data

[Uber's Orbit ML Package](https://medium.com/@alexzap922/bayesian-time-series-forecasting-in-python-with-the-ubers-orbit-package-1d3b7ff482dd)

Orbit currently supports the the following forecasting models:

- **Exponential Smoothing (ETS)**
- **Damped Local Trend (DLT)**
- **Local Global Trend (LGT)**

Orbit has a wide range of applications in Uber’s marketing data science team for measurement, planning, and forecasting. The latter is an important part of planning future marketing budgets and the optimization of spending across different channels and regions.

In [ ]:
!pip install -q numpy pandas matplotlib plotly
!pip install -q arviz orbit-ml

#### Grocery Store Sales Forecasting

The `Kaggle` dataset will be used to predict sales for the thousands of product families sold at Favorita stores located in Ecuador.

In [ ]:
import orbit
from orbit.models import DLT, ETS
from orbit.diagnostics.backtest import BackTester
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Auxiliary WMAPE function
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

path = 'train.csv'
data = pd.read_csv(path, index_col='id', parse_dates=['date'])

# General data structure
data.tail()

In [ ]:
data.info()

In [ ]:
# Selecting store_nbr == 1
data2 = data.loc[((data['store_nbr'] == 1)), ['date', 'unit_sales', 'onpromotion']]

# Selecting training and validation sets
dec25 = list()
for year in range(2013,2017):
    dec18 = data2.loc[(data2['date'] == f'{year}-12-18')]
    dec25 += [{'date': pd.Timestamp(f'{year}-12-25'), 'unit_sales': dec18['unit_sales'].values[0], 'onpromotion': dec18['onpromotion'].values[0]}]
data2 = pd.concat([data2, pd.DataFrame(dec25)], ignore_index=True).sort_values('date')


train = data2.loc[data2['date'] < '2017-01-01']
valid = data2.loc[(data2['date'] >= '2017-01-01') & (data2['date'] < '2017-04-01')]

df_daily = train.set_index('date').resample('D')["unit_sales"].sum().to_frame()

df_daily.tail()

##### Training the Bayesian Exponential Smoothing (ETS) model

In [ ]:
ets = ETS(date_col='date', 
          response_col='unit_sales', 
          seasonality=7,
          prediction_percentiles=[5, 95],
          seed=1)

ets.fit(df=df_daily)

In [ ]:
p = ets.predict(df=df_daily)

p.tail()

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(p['date'],p['prediction'])
plt.plot(p['date'],df_daily['unit_sales'])

In [ ]:
forecast_df = pd.DataFrame({"date": pd.date_range(start='2017-01-01', end='2017-03-31', freq='D')})
p = ets.predict(df=forecast_df)
p = p.merge(valid, on='date', how='left')

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize=(1280/96, 720/96))
ax.plot(p['date'], p['sales'], label='actual')
ax.plot(p['date'], p['prediction'], label='prediction')
ax.fill_between(p['date'], p['prediction_5'], p['prediction_95'], alpha=0.2, color='orange', label='prediction percentiles')
ax.set_title('Error, Trend, Seasonality (ETS)')
ax.set_ylabel('Sales')
ax.set_xlabel('Date')
ax.legend()
plt.show()

##### Training the Damped Local Trend (DLT) Model

In [ ]:
scores = dict()
for global_trend_option in ['linear', 'loglinear', 'flat', 'logistic']:
    dlt = DLT(date_col='date', 
            response_col='sales', 
            seasonality=7,
            prediction_percentiles=[5, 95],
            regressor_col=['onpromotion'],
            regressor_sign=['='],
            regression_penalty='auto_ridge',
            damped_factor=0.8,
            seed=2, # if you get errors due to less than zero values, try a different seed
            global_trend_option=global_trend_option,
            verbose=False)
    bt = BackTester(df=train, 
                    model=dlt, 
                    forecast_len=90,
                    n_splits=5,
                    window_type='rolling')

    bt.fit_predict()
    predicted_df = bt.get_predicted_df()

scores[global_trend_option] = wmape(predicted_df['actual'], predicted_df['prediction'])

print("Scores:\n", scores)

best_global_trend_option = min(scores, key=scores.get)
print("Best Global Trend: ", best_global_trend_option)

In [ ]:
df_daily = train.set_index('date').resample('D')["unit_sales"].sum().to_frame()

df_daily11 = train.set_index('date').resample('D')["onpromotion"].sum()

df_daily11.tail()

In [ ]:
df_daily1.set_index('date')

df_daily1['date1'] = df_daily1['date']

df_daily1.set_index('date1')

df_daily = pd.concat([df_daily, df_daily1["onpromotion"]], axis=1)

df = df_daily.reset_index()

df.tail()

In [ ]:
dlt = DLT(
    response_col='unit_sales',
    date_col='date',
    estimator='stan-map',
    seasonality=52,
    seed=8888,
    global_trend_option='logistic',
    # for prediction uncertainty
    n_bootstrap_draws=1000,
)

dlt.fit(df)
p1 = dlt.predict(df)

p1.tail()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(1280/96, 720/96))
ax.plot(p1['date'], df['unit_sales'], label='actual')
ax.plot(p1['date'], p1['prediction'], label='prediction')
ax.fill_between(p1['date'], p1['prediction_5'], p1['prediction_95'], alpha=0.2, color='orange', label='prediction percentiles')
ax.set_title('DLT Model')
ax.set_ylabel('Sales')
ax.set_xlabel('Date')
ax.legend()
plt.show()

In [ ]:
dlt = DLT(date_col='date', 
        response_col='sales', 
        seasonality=7,
        prediction_percentiles=[5, 95],
        regressor_col=['onpromotion'],
        regressor_sign=['='],
        regression_penalty='auto_ridge',
        damped_factor=0.8,
        seed=2, # if you get errors due to less than zero values, try a different seed
        global_trend_option=best_global_trend_option,
        verbose=False)

dlt.fit(df=train)

p = dlt.predict(df=valid[['date', 'onpromotion']])
p = p.merge(valid, on='date', how='left')

print(wmape(p['sales'], p['prediction']))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(1280/96, 720/96))
ax.plot(p['date'], p['sales'], label='actual')
ax.plot(p['date'], p['prediction'], label='prediction')
ax.fill_between(p['date'], p['prediction_5'], p['prediction_95'], alpha=0.2, color='orange', label='prediction percentiles')
ax.set_title('Damped Local Trend (DLT)')
ax.set_ylabel('Sales')
ax.set_xlabel('Date')
ax.legend()
plt.show()

In [ ]:
from orbit.models import KTR

ktr = KTR(date_col='date', 
        response_col='sales', 
        seasonality=[7, 28],
        prediction_percentiles=[5, 95],
        regressor_col=['onpromotion'],
        seed=2,
        verbose=False)

ktr.fit(df=train)

p = ktr.predict(df=valid[['date', 'onpromotion']])
p = p.merge(valid, on='date', how='left')

print(wmape(p['sales'], p['prediction']))

#### US Unemployment Benefits

In [ ]:
import pandas as pd
import numpy as np
import orbit
import matplotlib.pyplot as plt

from orbit.utils.dataset import load_iclaims
from orbit.diagnostics.plot import plot_predicted_data, plot_predicted_components
from orbit.utils.plot import get_orbit_style
plt.style.use(get_orbit_style())
from orbit.models import ETS

print(orbit.__version__)

In [ ]:
raw_df = load_iclaims(transform=True)
raw_df.dtypes

In [ ]:
df = raw_df.copy()
df.head()

In [ ]:
test_size=52

train_df=df[:-test_size]
test_df=df[-test_size:]

##### Training the ETS Model

In [ ]:
ets = ETS(
    response_col='claims',
    date_col='week',
    seasonality=52,
    seed=2020,
    estimator='stan-mcmc',
)

ets.fit(train_df)

predicted_df = ets.predict(df=df, decompose=True)
predicted_df

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df,
                        predicted_df=predicted_df,
                        date_col='week',
                        actual_col='claims',
                        test_actual_df=test_df)

In [ ]:
_ = plot_predicted_components(predicted_df=predicted_df, date_col='week')

In [ ]:
posterior_samples = ets.get_posterior_samples()
posterior_samples.keys()

In [ ]:
import arviz as az

posterior_samples = ets.get_posterior_samples(permute=False)

# example from https://arviz-devs.github.io/arviz/index.html
az.style.use("arviz-darkgrid")
az.plot_pair(
    posterior_samples,
    var_names=["sea_sm", "lev_sm", "obs_sigma"],
    kind="kde",
    marginals=True,
    textsize=15,
)
plt.show()

##### Training the Local Global Trend (LGT) Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import orbit
from orbit.models import LGT
from orbit.diagnostics.plot import plot_predicted_data
from orbit.diagnostics.plot import plot_predicted_components
from orbit.utils.dataset import load_iclaims

# load data
df = load_iclaims()
# define date and response column
date_col = 'week'
response_col = 'claims'
df.dtypes
test_size = 52
train_df = df[:-test_size]
test_df = df[-test_size:]

lgt = LGT(
    response_col=response_col,
    date_col=date_col,
    estimator='stan-map',
    seasonality=52,
    seed=8888,
)

lgt.fit(df=train_df)

predicted_df = lgt.predict(df=test_df)

_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df,
                        date_col=date_col, actual_col=response_col,
                        test_actual_df=test_df, title='Prediction with LGTMAP Model')

In [ ]:
#LGT - MCMC
lgt = LGT(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
)

In [ ]:
%%time
lgt.fit(df=train_df)

In [ ]:
predicted_df = lgt.predict(df=test_df)

predicted_df.tail(5)

In [ ]:
lgt.get_posterior_samples().keys()

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df,
                    date_col=lgt.date_col, actual_col=lgt.response_col,
                    test_actual_df=test_df, title='Prediction with LGTFull Model')

In [ ]:
#LGT - point estimate
lgt = LGT(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
)
lgt.fit(df=train_df, point_method='mean')
predicted_df = lgt.predict(df=test_df)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df,
                    date_col=lgt.date_col, actual_col=lgt.response_col,
                    test_actual_df=test_df, title='Prediction with LGTAggregated Model')

##### Training the DLT Regression Model

In [ ]:
response_col = 'claims'
date_col = 'week'
regressor_col = ['trend.unemploy', 'trend.filling', 'trend.job']

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import orbit
from orbit.models import DLT
from orbit.diagnostics.plot import plot_predicted_data,plot_predicted_components
from orbit.utils.dataset import load_iclaims

import warnings
warnings.filterwarnings('ignore')
print(orbit.__version__)

In [ ]:
# load log-transformed data
df = load_iclaims()
# train/test split
train_df = df[df['week'] < '2017-01-01']
test_df = df[df['week'] >= '2017-01-01']
# regression parameters
response_col = 'claims'
date_col = 'week'
regressor_col = ['trend.unemploy', 'trend.filling', 'trend.job']

In [ ]:
# training the model
dlt = DLT(
    response_col=response_col,
    regressor_col=regressor_col,
    date_col=date_col,
    seasonality=52,
    prediction_percentiles=[5, 95],
)

dlt.fit(train_df)

In [ ]:
# making in-sample predictions of the training set
predicted_df = dlt.predict(df=train_df, decompose=True)

# plotting in-sample predictions of the training set
_ = plot_predicted_data(train_df, predicted_df,
                        date_col=dlt.date_col, actual_col=dlt.response_col)

In [ ]:
predicted_df = dlt.predict(df=test_df, decompose=True)

_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df,
                        date_col=dlt.date_col, actual_col=dlt.response_col,
                        test_actual_df=test_df)

In [ ]:
predicted_df = dlt.predict(df=train_df, decompose=True)

_ = plot_predicted_components(predicted_df, date_col)

In [ ]:
_ = plot_predicted_components(predicted_df, date_col,
                              plot_components=['prediction', 'trend', 'seasonality', 'regression'])